In [0]:
import pandas as pd
import numpy as np
import sklearn
import nltk
import keras
import string as st
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from google.colab import files
from keras.models import Sequential
from keras.layers import Embedding
from keras.layers import RepeatVector
from keras.layers import LSTM
from keras.layers import Dense
from keras import optimizers
from nltk.translate.bleu_score import corpus_bleu

***Run the code in google colab.***

# ***Neural Machine Translation from German to english using encoder decoder sequence model.***


# ***Dataset is restricted to 35000 records apporx. for this execution (total dataset is of 174481 records).***

I have generated the output in the csv file 'predicted.csv' which will be visible in the left pane on refreshing.

In [16]:
# convert data into dataframe  
data = pd.read_csv('deu.txt', delimiter='\t', names=['English', 'German'], usecols=[0,1])
dataCopy=data
data.head()

,English,German
0,Hi.,Hallo!
1,Hi.,Grüß Gott!
2,Run!,Lauf!
3,Wow!,Potzdonner!
4,Wow!,Donnerwetter!


In [0]:
data.shape
# restrict dataset to 35000 records
dataLimit = np.random.rand(len(dataCopy)) <= 0.18
limitDataset=dataCopy[dataLimit]

In [18]:
limitDataset.shape

(36938, 2)

In [19]:
#keras based tokenizer is used to map words to integer
tokenizerEn=Tokenizer()
tokenizerFr=Tokenizer()

tokenizerEn.fit_on_texts(limitDataset['English'].to_numpy())
vocabSizeEn = len(tokenizerEn.word_index) + 1

tokenizerFr.fit_on_texts(limitDataset['German'].to_numpy())
vocabSizeFr = len(tokenizerFr.word_index) + 1

print('Vocabulary Size: English ', vocabSizeEn, "German ", vocabSizeFr)

Vocabulary Size: English  8654 German  15025


In [20]:
#Data preprocessing to clean the data and get the maximum sized length of sentences from both languages
maxLenEn = 0
maxLenFr = 0
for index, row in limitDataset.iterrows():
    row['English']=row['English'].translate(str.maketrans('', '', st.punctuation)).lower()
    if len(row['English'].split())>maxLenEn:
        maxLenEn=len(row['English'].split())
    row['German']=row['German'].translate(str.maketrans('', '', st.punctuation)).lower()
    if len(row['German'].split())>maxLenFr:
        maxLenFr=len(row['German'].split())  
print("Maximum sized length : English ",maxLenEn,"German ", maxLenFr )

Maximum sized length : English  38 German  41


In [21]:
# we will split the data to keep 70:30 ratio for training and test data
splitData = np.random.rand(len(limitDataset)) <= 0.7
trainData = limitDataset[splitData]
testData = limitDataset[~splitData]
#check the data count in training set and test set
print("Training Data: ",trainData.shape," Test data: ",testData.shape)
# Data in to numpy array conversion
trainDataNP=trainData.to_numpy()
testDataNP=testData.to_numpy()

Training Data:  (25882, 2)  Test data:  (11056, 2)


**Data encoding is done i.e. converting text to ids and add padding to make all sequences of same length. **

In [0]:
def dataEncoder(allData, lanTokenizer, length):
    encodedData = lanTokenizer.texts_to_sequences(allData)
    paddedData = pad_sequences(encodedData, maxlen=length, padding='post')
    return paddedData

trainFr = dataEncoder(trainDataNP[:, 1],tokenizerFr,maxLenFr)
trainEn = dataEncoder(trainDataNP[:, 0],tokenizerEn,maxLenEn)
testFr = dataEncoder(testDataNP[:, 1],tokenizerFr,maxLenFr)
testEn = dataEncoder(testDataNP[:, 0],tokenizerEn,maxLenEn)

In [23]:
print(trainFr.shape, testFr.shape)

(25882, 41) (11056, 41)


In [0]:
#Sequence model
def sequenceModel(vocabFr, vocabEn, maxFr, maxEn, units):
    seqModel = Sequential()
    seqModel.add(Embedding(vocabFr, units, input_length=maxFr, mask_zero=True))
    seqModel.add(LSTM(units))
    seqModel.add(RepeatVector(maxEn))
    seqModel.add(LSTM(units, return_sequences=True))
    seqModel.add(Dense(vocabEn, activation='softmax'))
    return seqModel

seqModel = sequenceModel(vocabSizeFr, vocabSizeEn, maxLenFr, maxLenEn, 256)
rmsOptimizer = optimizers.RMSprop(lr=0.01)
seqModel.compile(optimizer=rmsOptimizer, loss='sparse_categorical_crossentropy')

In [25]:
modelPrediction = seqModel.fit(trainFr, trainEn.reshape(trainEn.shape[0], trainEn.shape[1], 1),epochs=15, batch_size=256, validation_split = 0.2, verbose=1)

Train on 20705 samples, validate on 5177 samples
Epoch 1/15
20705/20705 [==============================] - 18s 867us/step - loss: 1.3919 - val_loss: 1.7848
Epoch 2/15
20705/20705 [==============================] - 16s 788us/step - loss: 0.8565 - val_loss: 2.0639
Epoch 3/15
20705/20705 [==============================] - 16s 785us/step - loss: 0.7567 - val_loss: 1.8454
Epoch 4/15
20705/20705 [==============================] - 16s 792us/step - loss: 0.6635 - val_loss: 1.9286
Epoch 5/15
20705/20705 [==============================] - 16s 788us/step - loss: 0.5785 - val_loss: 1.7782
Epoch 6/15
20705/20705 [==============================] - 16s 771us/step - loss: 0.5028 - val_loss: 1.8772
Epoch 7/15
20705/20705 [==============================] - 16s 788us/step - loss: 0.4365 - val_loss: 1.7390
Epoch 8/15
20705/20705 [==============================] - 16s 778us/step - loss: 0.3771 - val_loss: 1.8549
Epoch 9/15
20705/20705 [==============================] - 16s 783us/step - loss: 0.3239 - val_l

In [0]:
#Get predictions for text data 
testPrediction = seqModel.predict_classes(testFr.reshape((testFr.shape[0],testFr.shape[1])))

In [0]:
## data decoder Convert mapped words in to normal words
def wordConversion(word_index, tokenizer):
    for word, index in tokenizer.word_index.items():
        if index == word_index:
            return word
    return None
    
def dataDecoder(prediction):
    textPredicted=list()
    for sentence in prediction:
        sentenceConverted = list()
        for word_index in range(len(sentence)):
            word = wordConversion(sentence[word_index], tokenizerEn)
            if word_index > 0:
                if (word == wordConversion(sentence[word_index-1], tokenizerEn)) or (word == None):
                    sentenceConverted.append('')
                else:
                    sentenceConverted.append(word)

            else:
                if(word == None):
                    sentenceConverted.append('')
                else:
                    sentenceConverted.append(word)            

        textPredicted.append(' '.join(sentenceConverted))
        
    return textPredicted

In [0]:
#calling data decoder for the predictions
predictedText = dataDecoder(testPrediction)
predictedDataframe = pd.DataFrame({'German text': testData['German'],' Expected text' : testData['English'], 'Predicted text' : predictedText})

# ***Data after conversion from German to english i.e. both the expected and predicted english text***

In [29]:

predictedDataframe.head(10)

,German text,Expected text,Predicted text
20,iss auf,eat up,be up
51,das kommt nicht in frage,no way,that my
55,wirklich,really,really
69,sei fair,be fair,be up
111,troll dich,go away,keep you
114,mach ’ne fliege,go away,go away
137,gehen sie weg,go away,get you lost
141,leb wohl,goodbye,do you know love ...
148,hilf uns,help us,keep us
156,umarme tom,hug tom,tom


# ***Using corpus bleu function to calculate the Bleu score for data***

In [30]:
print('BLEU score using corpus bleu function:',corpus_bleu(testDataNP[:,0], predictedText))

BLEU score using corpus bleu function: 0.648951343411966


/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


## ***Generating the csv file for predicted output***



In [0]:
predictedDataframe.to_csv('predicted.csv')